# Westlake Realty Group Proposal

Authors: Drew Holcombe, Juan Acosta

## Overview
Westlake Realty Group has grown into a diversified, real estate company that specializes in property management, commercial real estate, and relocation services. Our project presents data of house sale prices for King County, Washington. Using linear regression and data analysis techniques we are able to find which factors are the most impactful to determine the value of a house, and how Westlake Realty Group could use this information to help them and their clients make decisions.
(hypothetical example)

## Business Understanding

As metropolitan areas keep growing in population, and the housing market is constantly changing, homebuyers could find themselves facing limited options. Westlake Realty Group is looking for factors and characteristics of a house that are the most impactful to determine its price, and be capable to help homebuyers manage their options based on specific features vs their budget.

Our group's objective is to find features that accurately predict the sell price, and provide results and justifications for Westlake Realty Group.



## Data Understanding
Our project presents data of around 21 thousand house-sale records in King county, Washington, contained in the kc_house dataset. It includes houses sold between May 2014 and May 2015. The variables we found to be most useful for our research were: `price`, `bedrooms`, `bathrooms`, `sqft_living`, `floors`, `waterfront`, `view`, `condition` and `grade`.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

In [ ]:
df = pd.read_csv('data/kc_house_data.csv')
df.shape

We have information of 21597 houses located in King county, Washington.

In [ ]:
df.info()

`price` will be our target for the model

<br>

For our numeric columns we have `bedrooms`, `bathrooms`, `sqft_living`, `floors`, `sqft_above`, `yr_built`, `yr_renovated`, `zipcode`, `lat`, `long`, `sqft_living15` and `sqft_lot15`.

Out of all numeric columns, `bedrooms`, `bathrooms` and `floors` contain discrete data, we can also notice how `sqft_basement` has an object dtype, we might have to convert values if the column is needed.

<br>

`waterfront`, `zipcode`, `view`, `condition` and `grade` are categorical variables
<br>

There are null values present in `waterfront`, `view` and `yr_renovated` columns

## Data Preparation

We first clean the data, look for duplicate data points, identify noticeable outliers, and dropping null values

In [ ]:
df.head(10)

First thing we can notice on the `sqft_basement` column, a `?` can be seen on some of the data points. 

<br>

#### `ID` column

`id` column shows the unique identifier for a house

In [ ]:
df['id'].value_counts()

We have duplicate values.

In [ ]:
# quick search for first id in the value count
df.loc[df["id"]==795000620]

Above we can see 3 sale records for the same house, price is different in the three occasions, increasing over time. We can also notice how the third row shows NaN value on the `waterfront` and `yr_renovated` columns, and that same house in a different row shows 'NO' and '0.0' respectively.

We dropped duplicate values using `id`, looks like duplicates are ordered by `date`, and by keeping the last data point we also keep the most recent record of the house.

In [ ]:
df.drop_duplicates(subset='id', keep='last', inplace=True)

In [ ]:
df.loc[df["id"]==795000620]

<br>


We dropped some columns to focus mainly on `price`, `bedrooms`, `bathrooms`, `sqft_living`, `sqft_lot`, `floors`, `waterfront`, `view`, `condition`, `grade` and `yr_built`

In [ ]:
df.drop(labels=["sqft_above", "sqft_basement", "yr_renovated", "zipcode", "lat",
                "long","sqft_living15","sqft_lot15"], axis=1, inplace=True)

<br>




Let's look for potential mistakes in other columns; We will define some functions for repeated use and see what we're working with.


#### `bedrooms` column

<br>


In [ ]:
def hist(df, column):
    return plt.hist(x=df[column], bins='auto')
def hist_rooms(df, column):
    return plt.hist(x=df[column], bins=np.linspace(1, df[column].max()))
def extreme(df, column):
    return df.loc[df[column] == df[column].max()]

In [ ]:
hist_rooms(df, "bedrooms");

In [ ]:
extreme(df, "bedrooms")

An outlier, a house with 33 bedrooms, if we do a quick search on google using the id number we'll see this is a typo, we will replace value with 3, the correct amount of rooms.

In [ ]:
df["bedrooms"].replace(to_replace=33, value=3, inplace=True)
df["bedrooms"].value_counts()

In [ ]:
df.loc[df["bedrooms"] == 11]

Looking into this house, it appears it actually has four bedrooms.

In [ ]:
df["bedrooms"].replace(to_replace=11, value=4, inplace=True)




<br>

#### `bathrooms` column

<br>


In [ ]:
hist_rooms(df, "bathrooms");

In [ ]:
extreme(df, "bathrooms")

The amount  of bathrooms in these houses can be justified by the size of them.



<br>


#### `sqft_living` column

In [ ]:
hist(df, "sqft_living");

In [ ]:
extreme(df, "sqft_living")

Understandably, the largest house also is tied for most bathrooms, but if we take a look at the plot below, this house  can be considered an outlier, it has a large sq ft living space and a low price, compared to houses similar in living space but sold for a higher price.

In [ ]:
plt.scatter(df['sqft_living'], df['price']);

In [ ]:
#dropping outlier

df.drop(12764, inplace = True)



<br>


#### `sqft_lot` column

In [ ]:
hist(df, "sqft_lot");

In [ ]:
extreme(df, "sqft_lot")

In [ ]:
df['sqft_lot'].max()

A id search reveals this is a farm, which is unusual.

In [ ]:
plt.scatter(df['sqft_lot'], df['price']);


<br>

#### `waterfront` column



The column tells whether the house is on a waterfront

In [ ]:
df['waterfront'].value_counts()

In [ ]:
df['waterfront'].isnull().sum()

Will change NaN values to 'NO' because it is the most frequent value under that column.

In [ ]:
df['waterfront'] = df['waterfront'].replace(np.nan, 'NO')

In [ ]:
plt.hist(df['waterfront']);


<br>


#### `floors` column

In [ ]:
df['floors'].value_counts()

In [ ]:
plt.hist(df['floors']);

In [ ]:
df.loc[df['floors'] == 3.5]

Made a search for house with 3.5 floors and only 2 bedrooms just to confirm the information is accurate, it is an apartment unit.

<br>


#### `conditions` column

<br>

The `condition` column tells us how good the overall condition of the house is, and is related to maintenance of house.

In [ ]:
plt.hist(df['condition']);

In [ ]:
plt.scatter(df['condition'], df['price']);

In [ ]:
dff = df.loc[df['condition'] == 'Good']
dff.loc[dff['price'] == dff['price'].max()]

<br>


#### `grade` column

<br>

The `grade` column shows the overall grade of the house, related to the construction and design of the house.

In [ ]:
df['grade'].value_counts()

In [ ]:
plt.hist(df['grade'])
plt.xticks(rotation = 60);

In [ ]:
plt.scatter(df['grade'], df['price'])
plt.xticks(rotation = 60);

We can notice the '13 mansion'  has some outliers, '11 Excellent' shows one outlier, we will take care of these

In [ ]:
Mansions = df.loc[df['grade'] == '13 Mansion']
Mansions.loc[Mansions['price'] == Mansions['price'].max()]

In [ ]:
df.drop(7245, inplace = True)

In [ ]:
df.drop(9245, inplace = True)

In [ ]:
Excellent = df.loc[df['grade'] == '11 Excellent']
Excellent.loc[Excellent['price'] == Excellent['price'].max()]

In [ ]:
df.drop(3910, inplace = True)

In [ ]:
plt.scatter(df['grade'], df['price'])
plt.xticks(rotation = 60);

<br>


#### `yr_built` column

<br>

Year when house was built

In [ ]:
hist(df, "yr_built");

We have records for houses built in 1900 to 2015

<br>


#### `view` column

<br>

This column describes quality of view from house

In [ ]:
df["view"].value_counts()

In [ ]:
df["view"].isna().sum()

Column view has 63 null values, we will replace those values and consider them as having no view, 'NONE'

In [ ]:
df['view'] = df['view'].replace(np.nan, 'NONE')


<br>

## Data Modeling

We defined functions to help create a simple model, and a second function to create models using a log transformed target.

In [ ]:
def simple_model(x):
    y = df["price"]
    x_c = sm.add_constant(x)
    return sm.OLS(endog=y, exog=x_c).fit().summary()

In [ ]:
def simple_ylog_model(x):
    y = np.log(df["price"])
    x_c = sm.add_constant(x)
    return sm.OLS(endog=y, exog=x_c).fit().summary()

Hot-coded values for view column

In [ ]:
view_dummies = pd.get_dummies(df["view"])
view_dummies
view_dummies.drop('NONE', axis=1, inplace=True)


<br>

Lets built a simple model using view features.

In [ ]:
simple_model(view_dummies)


<br>

Created a function to add dummy view columns to a single dataframe including `bedrooms`	`bathrooms`	`sqft_living`	`sqft_lot` features and set them as our exogenous.

In [ ]:
def add_views(df):
    df["Fair_view"] = view_dummies["FAIR"]
    df["Average_view"] = view_dummies["AVERAGE"]
    df["Good_view"] = view_dummies["GOOD"]
    df["Excellent_view"] = view_dummies["EXCELLENT"]

In [ ]:
data_d = df.drop(labels=["id","date","price","floors","waterfront","condition","grade","yr_built","view"], axis=1)
add_views(data_d)
data_d

In [ ]:
x, y = sm.add_constant(data_d), df["price"]

In [ ]:
sm.OLS(y, x).fit().summary()

<br>

Let's log tranform our target and see if it will improve our model.

<br>

In [ ]:
y_log = np.log(y)
sm.OLS(y_log, x).fit().summary()

While the  `bathrooms` feature is showing a stronger correlation with the log of the price, the other factors prevented this from being the best overall choice.

Let's scale this data

In [ ]:
x_nums = data_d.drop(labels=["Fair_view", "Average_view", "Good_view", "Excellent_view"], axis=1)

In [ ]:
x_scaled = StandardScaler().fit_transform(x_nums.values)
x_scaled_df = pd.DataFrame(x_scaled, index=x_nums.index, columns=x_nums.columns)
x_c = sm.add_constant(x_scaled_df)

In [ ]:
sm.OLS(y, x_c).fit().summary()


<br>

Now that our condition number is minimal, if it increases by adding the `view` features, we'll know those inputs are the ones causing the ill-condition. Let's add them in now!

In [ ]:
add_views(x_c)

In [ ]:
sm.OLS(y, x_c).fit().summary()

let's run the model using log y

In [ ]:
sm.OLS(y_log, x_c).fit().summary()

### Checking Assumptions

We assign latest model to a variable and proced to check assumptions

In [ ]:
model = sm.OLS(y_log, x_c).fit()

In [ ]:
model.summary().tables[2]

In [ ]:
model_preds = model.predict(x_c)
resids = y_log - model_preds

In [ ]:
fig, ax = plt.subplots()

ax.scatter(y, resids);

In [ ]:
model2 = sm.OLS(endog=y_log, exog=x_c).fit()
model2_preds = model2.predict(x_c)
model2_resid = y_log - model2_preds

fig, ax = plt.subplots()
ax.scatter(y_log, model2_resid);

In [ ]:
sm.qqplot(model2_resid, line='r');

Below we continued adding features to our model looking for a higher r-squared value while trying to keep the 
condition number low.

First we drop columns already used, and keep mosly categorical data.

In [ ]:
df2 = df.drop(labels=["id","date","price","bedrooms","bathrooms","sqft_living","sqft_lot","view"],axis=1)

In [ ]:
dummy_df = pd.get_dummies(df2)

In [ ]:
dummy_df.drop(labels=["waterfront_NO","condition_Average","grade_7 Average"],axis=1,inplace=True)

 `waterfront_NO`,`condition_Average`,`grade_7 Average` will be our reference categories.

In [ ]:
x_c_comb = dummy_df.join(x_c)

In [ ]:
sm.OLS(y_log, x_c_comb).fit().summary()

The high conditional number isn't ideal,  let's see if we can identify what's causing it by dropping a column. `Year_built` is the most continuous variable that hasn't been scaled, so let's try dropping it as a test.

In [ ]:
test = x_c_comb.drop(labels="yr_built",axis=1)

In [ ]:
sm.OLS(y_log, test).fit().summary()

That dropped our conditional number dramatically! 

<br>

Let's scale the year built column and add it back in.

In [ ]:
yr_scaled = (x_c_comb["yr_built"] - np.mean(x_c_comb["yr_built"])) / np.std(x_c_comb["yr_built"])
yr_scaled_df = pd.DataFrame(yr_scaled, index=x_c_comb["yr_built"].index)

In [ ]:
x_c_comb2 = test.merge(yr_scaled_df,left_index=True, right_index=True)

In [ ]:
sm.OLS(y_log, x_c_comb2).fit().summary()

In [ ]:
print(f"Square Foot of Living Space std: {df['sqft_living'].std()}")
print(f"Square Foot of Lot std: {df['sqft_lot'].std()}")
print(f"Year built std: {df['yr_built'].std()}")
print(f"Bedroom std: {df['bedrooms'].std()}")
print(f"Bathroom std: {df['bathrooms'].std()}")

# Conclusions

#### Regression Results

Our model suggests the most impactful features to predict house prices within king county are waterfront, size of living area, the construction quality and view. We found that year built and bedrooms do not have as much of an impact as it could be expected.

Some key points to mention are:

- Adding a floor ups the price by 7.6%

- Being waterfront increases the price by 30%

- Adding a 3/4 bathroom increases price by 6%

- Adding 909 square feet to living area increases price by 17%

- Having a view can increase the price from 17 to 26%

- A home in poor condition can be nearly 13% cheaper, while a very good home can be 8% more expensive

- Construction quality has a massive impact; below-average homes sell 27 to 42% cheaper than average homes, while more luxury homes sell for as much as 88% more

- Year built, square footage of the lot, and number of bedrooms do not have significant impacts.

#### Limitatios and Next steps
Our resource only included records from 12 months (may 2014 - may 2015), and are from seven years ago, the housing market changes constantly and certainly more up to date data could result in interesting predictions.

The dataset included details on a small fraction of houses were renovations got done, but it was far from being substantial enough to put it in a model, by researching more data about it, we could find the impact it has on price, and predict if its something you could use in favor of increasing profit or client satisfaction. In addition to, something to further explore is how other factors such as new developments in a certain area could inflict or benefit the value of a property, development for hospitals, open spaces, public buildings, or even the school district.

##### Graphs for presentation

In [ ]:
import matplotlib.ticker as mtick

y = df["price"]
x = df["sqft_living"]
fig, ax = plt.subplots()
ax.scatter(x=x, y=y, c="olivedrab")
ax.ticklabel_format(style='plain')
ax.set_title("Sale Price vs. Square Footage of Home", size=20)
ax.set_ylabel("Sale Price", size=14)
ax.set_xlabel("Square Footage of Home", size=14)
fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
ax.yaxis.set_major_formatter(tick);
# plt.savefig("./SquareFootage.png", bbox_inches="tight", transparent=True, dpi=600);

In [ ]:
grades = ['3 Poor', '4 Low', '5 Fair', '6 Low Average', '7 Average', '8 Good', '9 Better', '10 Very Good',
          '11 Excellent', '12 Luxury', '13 Mansion']
grade_med_price = []
for x in grades:
    grade_med_price.append(df.loc[df["grade"] == x]["price"].median())

In [ ]:
plt.style.use('seaborn')
fig, ax = plt.subplots()

palette =['olivedrab', 'olivedrab', 'olivedrab', 'olivedrab', 'black', 'olivedrab', 'olivedrab',
                             'olivedrab', 'olivedrab', 'olivedrab', 'olivedrab']

ax.bar(x=grades, height=grade_med_price, color=palette)

ax.set_title("Median Sale Price by Home Grade", size=20)
ax.set_xlabel("Grade of Home", size=14)
ax.set_ylabel("Median Sale Price", size=14)
plt.ticklabel_format(axis="y", style='plain')
ax.tick_params(axis='x', labelrotation = 70)

fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
ax.yaxis.set_major_formatter(tick) 

# plt.savefig("./HomeGrade2.png", bbox_inches="tight", transparent=True, dpi=600);